# Measure Limbs

The aim of this notebook is to automatically take the same measurements as in the 1D dataset on the limbs created using the SSM. What this is going to lead to is the creation of a ML pipeline which will take in measurements and output the components of the SSM that get closest to that particular set of measurements (Also lets include an output for scale). Inshallah.

I think the approach I'm going to take for the circumference measurements is fairly simple - define a plane based on a vertical normal and a selected vertex. For every vert figure out which side of the plane its on. Find edges which cut the plane (And vertices on the plane). Find the intersection points between the edge and the plane. Join points based on face adjaceny - et voila. A curve on the plane. Then its just a matter of adding up all the line segments in a loop and bobs your uncle you have the circumfrential measurement. Maybe there is a nicer way to do it with a half edge mesh but I really haven't played about with that too much. 

As for the lengths: Just pick to verts and off you go. Nothing complicated about that at all.

In [1]:
import igl
import numpy

verts, face2vert = igl.read_triangle_mesh("stls/limb_00000.stl")